# Snow cover at selected points (Rosalia dataset)

This is a notebook that show-cases the usage of snowMAUS at hand of the Rosalia
dataset.

## Loading the dataset

The dataset is provided in the CSV format. The package pandas conveniently
handles data in table formats and can be used for the current dataset.

While, usually, pandas handles decoding files well, in this case one needs to
specify the file's encoding (encoding="latin").

*note*  
Its good practice to view data using a basic editor. In this case, for
example, the files include information about the data units and the measuring
location.

In [ ]:
import pandas as pd

In [ ]:
precip = pd.read_csv(
    "RR_Rosalia.csv",
    encoding="latin",
    index_col=0,  # use first column as index
    header=0,  # use first row as column names
    skiprows=[1,2,3],  # do not read second until forth row
    parse_dates=True  # convert strings to dates if possible
).rename_axis("time")  # name index "time"
print("Daily precipitation data in kg/m²:\n", precip)

In [ ]:
T_min = pd.read_csv("TN_Rosalia.csv", encoding="latin", index_col=0, header=0,
                    skiprows=[1,2,3], parse_dates=True).rename_axis("time")
print("Daily temperature minimum in ˚C:\n", T_min)

In [ ]:
T_max = pd.read_csv("TX_Rosalia.csv",  # this file uses the conventional utf-8 encoding
                    index_col=0, header=0, skiprows=[1,2,3], parse_dates=True
                    ).rename_axis("time")
print("Daily temperature maximum in ˚C:\n", T_max)

## Calculate the snow cover

With the data available, one can use snowMAUS to estimate the snow cover.

In [ ]:
# load snowMAUS. assuming it is in the parent directory, append it to the search
# path first
import sys
sys.path.append("..")
import snowmaus

In [ ]:
import numpy as np

In [ ]:
def snowcover(prev_day, accum, melt):
    balance = prev_day + accum - melt
    return np.where(balance > 0, balance, 0)

The commented function is easier to read but takes about twice as long as the
one following below the commented lines.

In [ ]:
# # iteratively calculate the snow cover
# # this can be done faster, if one pre-calculates the accumulation and the potential melt
# snowcover_df = pd.DataFrame(index=precip.index, columns=precip.columns)
# snowcover_df.iloc[0] = snowcover(0, snowmaus.accumulation(precip.iloc[0], T_min.iloc[0]), snowmaus.melt(T_min.iloc[0], T_max.iloc[0]))
# for i in range(1, len(precip.index)):
#     snowcover_df.iloc[i] = snowcover(snowcover_df.iloc[i-1], snowmaus.accumulation(precip.iloc[i], T_min.iloc[i]), snowmaus.melt(T_min.iloc[i], T_max.iloc[i]))
# snowcover_df

In [ ]:
daily_accum = pd.DataFrame(
    data=snowmaus.accumulation(precip, T_min),
    index=precip.index,
    columns=precip.columns
)
print("Daily snow accumulation in the same unit as \"precip\":\n", daily_accum)

In [ ]:
# the potential melt is the amount of snow that could be melted if it was
# present
potential_melt = pd.DataFrame(
    data=snowmaus.melt(T_min, T_max),
    index=T_min.index,
    columns=T_min.columns
)
print("Potential meltwater runoff in mm:\n", daily_accum)

*note*  
The units mm and kg/m² are equivalent when used with water (density = 1000 kg/m³).

In [ ]:
snowcover_df = pd.DataFrame(index=precip.index, columns=precip.columns)
snowcover_df.iloc[0] = snowcover(0, daily_accum.iloc[0], potential_melt.iloc[0])
for i in range(1, len(precip.index)):
    snowcover_df.iloc[i] = snowcover(snowcover_df.iloc[i-1], daily_accum.iloc[i], potential_melt.iloc[i])
snowcover_df

In [ ]:
reference = pd.read_csv("swe_tot_Rosalia.csv", encoding="latin", index_col=0, header=0,
                        skiprows=[1,2,3], parse_dates=True).rename_axis("time")
print("Snow cover in kg/m²:\n", reference)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# function to quickly view the modelled data next to the observations
def visual_comparison(model, reference, year: int = 2023, location_name: str = "Forchtenstein"):
    model.loc[f"{year}-10":f"{year+1}-04",location_name].plot(label="model")
    reference.loc[f"{year}-10":f"{year+1}-04",location_name].plot(label="observations")
    plt.legend()
    plt.ylabel("Snow cover in kg/m²")
    plt.xlabel("")
    plt.title(location_name)
    plt.show()

In [ ]:
visual_comparison(snowcover_df, reference)